# 📊 Quant Risk Lab: Hybrid Volatility Forecasting

**Author:** Alvaro Macias, PhD  
**Objective:** Benchmark Classical Econometrics (GARCH) against Deep Learning (LSTM) for Financial Risk Management.

## 1\. Theoretical Foundation

### 1.1 The Problem: Latent Volatility

In finance, volatility ($\sigma_t$) is not directly observable. We only see prices ($P_t$). To train any model, we first need a statistical proxy for "true" variance.
We use **Squared Log Returns** as our unbiased (noisy) estimator for daily variance:

$$r_t = \ln(\frac{P_t}{P_{t-1}})$$
$$\hat{\sigma}^2_t \approx r_t^2$$

### 1.2 Stationarity

Most time-series models (GARCH) assume **Stationarity** (mean and variance do not drift forever). Prices are *not* stationary, but Log Returns usually *are*. We verify this using the **Augmented Dickey-Fuller (ADF)** test.

  * **Null Hypothesis ($H_0$):** A Unit Root is present (Non-Stationary).
  * **Alternate Hypothesis ($H_1$):** The series is Stationary.
  * **Threshold:** We need p-value $< 0.05$ to proceed.

-----

## 2\. Step 1: Data Ingestion & Validation

*We implement the pipeline to fetch data and prove it is model-ready.*


In [ ]:
# Setup & Imports
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# FUNCTION: Data Pipeline
# ---------------------------------------------------------
def get_market_data(ticker='SPY', start='2015-01-01'):
    print(f"Fetching data for {ticker}...")
    df = yf.download(ticker, start=start, progress=False)
    
    # 1. Log Returns
    df['log_ret'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
    
    # 2. Target: Squared Returns (Proxy for Variance)
    df['target_variance'] = df['log_ret'] ** 2
    
    df.dropna(inplace=True)
    return df

# Execute
data = get_market_data('SPY')

# ---------------------------------------------------------
# VALIDATION: Stationarity Test
# ---------------------------------------------------------
result = adfuller(data['log_ret'])
print(f"ADF Statistic: {result[0]:.4f}")
print(f"p-value: {result[1]:.4f}")

if result[1] < 0.05:
    print("✅ PASS: Series is Stationary.")
else:
    print("❌ FAIL: Series is Non-Stationary (Diff required).")

# Plot
plt.figure(figsize=(10, 4))
plt.plot(data['log_ret'], alpha=0.5)
plt.title(f"Log Returns of SPY")
plt.show()


## 3\. Step 2: The Benchmark - GARCH(1,1)

### 3.1 The Math

The **Generalized Autoregressive Conditional Heteroskedasticity (GARCH)** model assumes that today's variance is a weighted average of:

1.  **Long-run Variance ($\omega$)**
2.  **Yesterday's Shock ($\alpha \epsilon^2_{t-1}$)**: The "News" component.
3.  **Yesterday's Variance ($\beta \sigma^2_{t-1}$)**: The "Memory" component.

$$\sigma^2_t = \omega + \alpha \epsilon^2_{t-1} + \beta \sigma^2_{t-1}$$

**Stability Constraint:** $\alpha + \beta < 1$. If this sums to $>1$, the variance explodes, and the model is invalid.

### 3.2 Implementation & "Quant Audit"

We train the model and then run **Meucci's Invariants Check** to ensure the residuals are white noise.


In [ ]:
from arch import arch_model
from scipy import stats
from statsmodels.stats.diagnostic import acorr_ljungbox

# ---------------------------------------------------------
# MODEL: GARCH Training
# ---------------------------------------------------------
# Scale returns by 100 for optimizer stability
returns_scaled = data['log_ret'] * 100 

# Define GARCH(1,1) with Normal distribution
model_garch = arch_model(returns_scaled, vol='Garch', p=1, o=0, q=1, dist='Normal')
res_garch = model_garch.fit(disp='off')

print(res_garch.summary())

# ---------------------------------------------------------
# DIAGNOSTIC: Meucci's Checklist
# ---------------------------------------------------------
print("\n--- MODEL AUDIT ---")

# 1. Stationarity Check
params = res_garch.params
persistence = params['alpha[1]'] + params['beta[1]']
print(f"Persistence (alpha+beta): {persistence:.4f} (< 1.0 is Stable)")

# 2. Independence of Residuals (Ljung-Box)
# We test squared standardized residuals for remaining clusters
std_resid = res_garch.std_resid
lb_test = acorr_ljungbox(std_resid**2, lags=[10], return_df=True)
lb_p = lb_test['lb_pvalue'].values[0]

print(f"Ljung-Box p-value: {lb_p:.4f} (> 0.05 indicates White Noise)")
if lb_p > 0.05:
    print("✅ PASS: Residuals are independent.")
else:
    print("⚠️ WARNING: Residuals still have patterns (Model Underfitted).")

## 4\. Step 3: The Challenger - LSTM with QLIKE Loss

### 4.1 Why MSE fails for Risk

Standard Mean Squared Error (MSE) treats over-prediction and under-prediction symmetrically.

  * **Over-predicting Vol:** You hedge too much (Costly).
  * **Under-predicting Vol:** You blow up (Fatal).

We use **QLIKE (Quasi-Likelihood)** loss, derived from the likelihood of a Gaussian process. It penalizes under-estimation heavily.

$$Loss = \ln(h_t) + \frac{r_t^2}{h_t}$$

  * $h_t$: Predicted Variance
  * $r_t^2$: Realized Variance

### 4.2 Implementation

We implement the custom loss and the LSTM network using PyTorch.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# ---------------------------------------------------------
# THEORY: The Custom Loss
# ---------------------------------------------------------
class QLIKE_Loss(nn.Module):
    def forward(self, pred_var, target_sq_ret):
        # Loss = log(h) + y^2/h
        loss = torch.log(pred_var) + (target_sq_ret / pred_var)
        return torch.mean(loss)

# ---------------------------------------------------------
# MODEL: LSTM Architecture
# ---------------------------------------------------------
class VolatilityLSTM(nn.Module):
    def __init__(self):
        super(VolatilityLSTM, self).__init__()
        # Input: 1 feature (Log Return)
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=1, batch_first=True)
        self.fc = nn.Linear(64, 1)
        self.activation = nn.Softplus() # Ensures Variance > 0

    def forward(self, x):
        out, _ = self.lstm(x)
        last_step = out[:, -1, :] # We only want the forecast for T+1
        var_pred = self.fc(last_step)
        return self.activation(var_pred)

# ---------------------------------------------------------
# PIPELINE: Training Loop
# ---------------------------------------------------------
# Prepare Data Sequences
seq_length = 60
X_list, y_list = [], []
ret_vals = (data['log_ret'].values * 100) # Scaled

for i in range(len(ret_vals) - seq_length):
    X_list.append(ret_vals[i:i+seq_length])
    y_list.append(ret_vals[i+seq_length]**2) # Target is NEXT day variance

X_t = torch.tensor(X_list, dtype=torch.float32).unsqueeze(-1)
y_t = torch.tensor(y_list, dtype=torch.float32)

# Init Model
model_lstm = VolatilityLSTM()
criterion = QLIKE_Loss()
optimizer = optim.Adam(model_lstm.parameters(), lr=0.002)

print("Training LSTM...")
losses = []
for epoch in range(30): # Short run for demo
    optimizer.zero_grad()
    outputs = model_lstm(X_t)
    loss = criterion(outputs.squeeze(), y_t)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

plt.plot(losses)
plt.title("QLIKE Loss Descent")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

## 5\. Step 4: The Showdown (Backtest)

We perform a rolling Out-of-Sample (OOS) evaluation.

  * **GARCH:** Analytical forecast.
  * **LSTM:** Neural inference.

We compare them using **RMSE** (Accuracy) and **QLIKE** (Risk Safety).

In [ ]:
# ---------------------------------------------------------
# EVALUATION: Generate Forecasts
# ---------------------------------------------------------
# 1. LSTM Forecasts (In-Sample for simplicity of demo)
model_lstm.eval()
with torch.no_grad():
    pred_var_scaled = model_lstm(X_t).squeeze().numpy()

# Rescale back to decimal variance
pred_var_lstm = pred_var_scaled / (100**2)
pred_vol_lstm = np.sqrt(pred_var_lstm) * np.sqrt(252) # Annualize

# 2. GARCH Forecasts (Conditional Volatility from fit)
# Align indices: GARCH returns 'cond_vol' for the same T. 
# We need to slice to match LSTM's "sequence length" loss
garch_vol_raw = res_garch.conditional_volatility[seq_length:] 
garch_vol_ann = (garch_vol_raw / 100) * np.sqrt(252)

# 3. The "Truth" (Realized Volatility Proxy)
# We use a 21-day rolling window of squared returns for the "Truth" plot
# to make it readable (pure squared returns are too noisy to view)
true_sq_returns = data['log_ret'].values[seq_length:]**2
true_vol_proxy = np.sqrt(pd.Series(true_sq_returns).rolling(21).mean()) * np.sqrt(252)

# ---------------------------------------------------------
# VISUALIZATION
# ---------------------------------------------------------
plt.figure(figsize=(14, 6))

# Plot Truth
plt.plot(true_vol_proxy.values, color='black', label='Realized Vol (21d smooth)', linewidth=2)

# Plot GARCH
plt.plot(garch_vol_ann.values, color='red', label='GARCH(1,1)', alpha=0.7)

# Plot LSTM
plt.plot(pred_vol_lstm, color='blue', label='Deep LSTM (QLIKE)', alpha=0.7)

plt.title("GARCH vs LSTM: Volatility Forecasting Comparison")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Conclusion

By observing the plot:

1.  **GARCH** tends to react smoothly and is mean-reverting.
2.  **LSTM** (with QLIKE) should react more aggressively to sudden spikes, as the loss function penalizes "missing" a crash.